# Amplitude embedding (quantum hardware)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on real quantum hardware --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [1]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [3]:
#image of the sky filled with double precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[5.3361439e+01+9.4710060e+01j 5.3361439e+01+9.4710060e+01j
  6.9706752e-03+1.8015926e-03j 5.3361439e+01+9.4710060e+01j]
 [5.3361439e+01+9.4710060e+01j 1.9618436e-03+3.9578325e-05j
  9.8187849e-03+1.8523181e-03j 4.1484777e-03+8.8834642e-03j]
 [5.0806468e-03+1.8292258e-03j 6.0321023e-03+1.6359966e-03j
  6.2760897e-03+5.2735382e-03j 5.6739193e-03+3.2265356e-03j]
 [5.3361439e+01+9.4710060e+01j 4.1771131e-03+7.2088479e-03j
  4.7120410e-03+7.2031408e-03j 4.4660508e-03+9.1231782e-03j]]


## 2) Applying a 2D FT (Fourier domain)

In [4]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 266.86652 +473.5984j    160.04927 +284.118j      53.367836 +94.6979j
   160.07397 +284.11377j ]
 [ 160.05547 +284.11743j    53.364502 +94.7241j    -53.352062 -94.71814j
    53.357525 +94.70952j ]
 [  53.362186 +94.689514j  -53.345905 -94.70908j  -160.07747 -284.10992j
   -53.364243 -94.703415j]
 [ 160.08098 +284.1226j     53.350006 +94.70003j   -53.35618  -94.70286j
    53.35062  +94.713165j]]


#### Sanity check

In [5]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[5.3361439e+01+9.4710060e+01j 5.3361439e+01+9.4710060e+01j
  6.9701672e-03+1.8005371e-03j 5.3361439e+01+9.4710060e+01j]
 [5.3361439e+01+9.4710060e+01j 1.9624233e-03+4.1484833e-05j
  9.8190308e-03+1.8513203e-03j 4.1501522e-03+8.8863373e-03j]
 [5.0818920e-03+1.8329620e-03j 6.0324669e-03+1.6362667e-03j
  6.2763691e-03+5.2738190e-03j 5.6738853e-03+3.2274723e-03j]
 [5.3361439e+01+9.4710060e+01j 4.1768551e-03+7.2097778e-03j
  4.7111511e-03+7.2009563e-03j 4.4667721e-03+9.1252327e-03j]]

Reals MSE: 4.1250152e-13
Imaginaries MSE: 2.4278377e-12


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [18]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires, shots=1)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

ValueError: probabilities do not sum to 1

## 4) Applying 2D IFT (fidelity test)

In [15]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[6.0586487e+01+9.7869766e+01j 6.0586487e+01+9.7869766e+01j
  3.3996105e-03+2.9029846e-03j 6.0586483e+01+9.7869774e+01j]
 [6.0586487e+01+9.7869774e+01j 4.2858124e-03+5.6741238e-03j
  3.0598640e-03+7.4594021e-03j 5.5594444e-03+7.5554848e-04j]
 [7.9100132e-03+8.5000992e-03j 7.3049068e-03+1.4829636e-04j
  9.7513199e-05+5.6180954e-03j 6.2329769e-03+7.7996254e-03j]
 [6.0586483e+01+9.7869774e+01j 6.4988136e-03+6.9434643e-03j
  9.1147423e-03+3.4153461e-03j 6.8483353e-03+5.4905415e-03j]]

Reals MSE: 7.3972365e-12
Imaginaries MSE: 1.12596816e-11


## Utils

### Quantum

### Classical

In [2]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask